In [1]:
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda, RunnableConfig
import asyncio


async def slow_thing(some_input: str, config: RunnableConfig) -> str:
    """Do something that takes a long time."""
    await asyncio.sleep(1) # Placeholder for some slow operation
    await adispatch_custom_event(
        "progress_event",
        {"message": "Finished step 1 of 3"},
        config=config # Must be included for python < 3.10
    )
    await asyncio.sleep(1) # Placeholder for some slow operation
    await adispatch_custom_event(
        "progress_event",
        {"message": "Finished step 2 of 3"},
        config=config # Must be included for python < 3.10
    )
    await asyncio.sleep(1) # Placeholder for some slow operation
    return "Done"

slow_thing = RunnableLambda(slow_thing)

async for event in slow_thing.astream_events("some_input", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': 'some_input'}, 'name': 'slow_thing', 'tags': [], 'run_id': '9076b973-4ba6-45d1-855e-32217bbccf1f', 'metadata': {}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': '9076b973-4ba6-45d1-855e-32217bbccf1f', 'name': 'progress_event', 'tags': [], 'metadata': {}, 'data': {'message': 'Finished step 1 of 3'}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': '9076b973-4ba6-45d1-855e-32217bbccf1f', 'name': 'progress_event', 'tags': [], 'metadata': {}, 'data': {'message': 'Finished step 2 of 3'}, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': '9076b973-4ba6-45d1-855e-32217bbccf1f', 'name': 'slow_thing', 'tags': [], 'metadata': {}, 'data': {'chunk': 'Done'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': 'Done'}, 'run_id': '9076b973-4ba6-45d1-855e-32217bbccf1f', 'name': 'slow_thing', 'tags': [], 'metadata': {}, 'parent_ids': []}
